In [1]:
#automatically reload stuff
%load_ext autoreload
%autoreload 2

In [2]:
#This is the scripts I have to import the data,
#save it as a intermediate json (patient_organ_data)
#then denoise that and save it as a second json (patient_organ_data_denoise)
#final version includes inputed values and the original mask

In [3]:
import Utils
import matplotlib.pyplot as plt
from SpatialPreprocessing import *
import pprint
import Formatting
from Constants import Const
import copy
import Models

ModuleNotFoundError: No module named 'simplejson'

In [4]:
#read in the organ info and save it to a default format

spatial_files = load_spatial_files()
od = CamprtOrganData()
##for pid,files in spatial_files.items():
##    test1 = od.read_spatial_file(files['doses'])
##    test2 = od.read_spatial_file(files['distances'])
##     print(pid, test1.columns, test2[OrganData.roi_cols].values.ravel().tolist())
pdict = od.process_cohort_spatial_dict(spatial_files)
Utils.np_dict_to_json(pdict,Const.processed_organ_json)

del pdict

NameError: name 'load_spatial_files' is not defined

In [59]:
#this will be the format of the future data that is assumed for other functions
#before transforming them into usable arrays
#currently missing values are Nan.  Organs preserves the order in other arrays
pdata = load_pdict()
# pdata
for pid,pentry in pdata['patients'].items():
    for o,oentry in pentry.items():
        for k,v in oentry.items():
            if (not Utils.iterable(v) and v <= 0):
                print(pid,o,k,v)

31 Lt_Anterior_Seg_Eyeball volume 0.0
31 Lt_Anterior_Seg_Eyeball mean_dose 0.0
31 Lt_Posterior_Seg_Eyeball volume 0.0
31 Lt_Posterior_Seg_Eyeball mean_dose 0.0
31 Rt_Posterior_Seg_Eyeball volume 0.0
31 Rt_Posterior_Seg_Eyeball mean_dose 0.0
31 Rt_Anterior_Seg_Eyeball volume 0.0
31 Rt_Anterior_Seg_Eyeball mean_dose 0.0
37 Lt_Anterior_Seg_Eyeball volume 0.0
37 Lt_Anterior_Seg_Eyeball mean_dose 0.0
37 Lt_Posterior_Seg_Eyeball volume 0.0
37 Lt_Posterior_Seg_Eyeball mean_dose 0.0
37 Rt_Posterior_Seg_Eyeball volume 0.0
37 Rt_Posterior_Seg_Eyeball mean_dose 0.0
37 Rt_Anterior_Seg_Eyeball volume 0.0
37 Rt_Anterior_Seg_Eyeball mean_dose 0.0
105 Lt_Anterior_Seg_Eyeball volume 0.0
105 Lt_Anterior_Seg_Eyeball mean_dose 0.0
105 Lt_Posterior_Seg_Eyeball volume 0.0
105 Lt_Posterior_Seg_Eyeball mean_dose 0.0
105 Rt_Posterior_Seg_Eyeball volume 0.0
105 Rt_Posterior_Seg_Eyeball mean_dose 0.0
105 Rt_Anterior_Seg_Eyeball volume 0.0
105 Rt_Anterior_Seg_Eyeball mean_dose 0.0
112 Lt_Anterior_Seg_Eyeball volu

In [60]:
#so this should check that the values make sense
#currently it implies there is an issue but I don't know what
def get_num_nans_per_organ(sdata):
    #should return a list of the % of patients missing data for each organ
    #for each entry type
    olist = sdata['organs']
    oarlist = olist + ['gtv']
    allnan = {}
    for key in ['distances','volume','centroids','mean_dose']:
        nancount = {o: 0 for o in oarlist}
        arr = Formatting.merged_spatial_array(sdata, key)
        for i,aa in enumerate(arr):
            #find missing organs for individual
            all_nan = np.argwhere(np.isnan(aa).all(axis=1))
            nanset = set([])
            if len(all_nan) < 1:
                continue
            for arg in all_nan[0]:
                nanorgan = oarlist[arg]
                nancount[nanorgan] = nancount[nanorgan] + 1
        nancount = [(k,np.round(100*nancount[k]/arr.shape[0],4)) for k,v in nancount.items() if v > 0]
        nancount = sorted(nancount, key = lambda x: -x[1])
        
        allnan[key] = nancount
    return allnan
get_num_nans_per_organ(pdata)

{'distances': [('Spinal_Cord', 1.81),
  ('Lt_Posterior_Seg_Eyeball', 1.3575),
  ('Rt_Brachial_Plexus', 0.905),
  ('Thyroid_cartilage', 0.905),
  ('Lt_Parotid_Gland', 0.905),
  ('Rt_Ant_Digastric_M', 0.905),
  ('Rt_Parotid_Gland', 0.4525),
  ('Rt_Lateral_Pterygoid_M', 0.4525),
  ('Lt_Mastoid', 0.4525),
  ('Hyoid_bone', 0.4525),
  ('Rt_Posterior_Seg_Eyeball', 0.4525),
  ('Lt_Anterior_Seg_Eyeball', 0.4525),
  ('Rt_Anterior_Seg_Eyeball', 0.4525),
  ('Glottic_Area', 0.4525)],
 'volume': [('Spinal_Cord', 1.81),
  ('Lt_Posterior_Seg_Eyeball', 1.3575),
  ('Esophagus', 0.905),
  ('Rt_Brachial_Plexus', 0.905),
  ('Thyroid_cartilage', 0.905),
  ('Lt_Parotid_Gland', 0.905),
  ('Rt_Ant_Digastric_M', 0.905),
  ('Cricoid_cartilage', 0.4525),
  ('Rt_Parotid_Gland', 0.4525),
  ('Rt_Lateral_Pterygoid_M', 0.4525),
  ('Lt_Mastoid', 0.4525),
  ('Hyoid_bone', 0.4525),
  ('Rt_Posterior_Seg_Eyeball', 0.4525),
  ('Lt_Anterior_Seg_Eyeball', 0.4525),
  ('Rt_Anterior_Seg_Eyeball', 0.4525),
  ('Glottic_Area', 0.45

In [68]:
#converts the json to a dictionary of arrays with all missing values imputed
#denoise alpha is the coefficient of how much of the densising is done 
#using the autoencoder used to input missing values [0 = none, 1 = all]
di = Formatting.DataInputer(denoise_alpha = 0)
ddict = di.get_formatted_arrays(pdata,retrain=True)
[(k,v.shape) for k,v in ddict.items()]

training stopped on epoch 2870
OrganAutoEncoder(
  (hidden_layers): Sequential(
    (0): Linear(in_features=2397, out_features=200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=200, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=20, bias=True)
    (5): Dropout(p=0.2, inplace=False)
    (6): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): Linear(in_features=20, out_features=100, bias=True)
    (9): ReLU()
    (10): Linear(in_features=100, out_features=200, bias=True)
    (11): ReLU()
    (12): Linear(in_features=200, out_features=2397, bias=True)
    (13): LeakyReLU(negative_slope=0.1)
  )
  (dropout_layer): Dropout(p=0.5, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)
tensor(0.1228, grad_fn=<MeanBackward0>)
distances
260.0000013049077 -24.640625 260.0 -24.640625
260.0 -24.640625 

mean_dose
86.25 -1.7750456660081682e-09 86.25 0.0
86.25 0.0 

centroids
658.3561 

[('distances', (221, 47, 46)),
 ('distances_missing', (221, 47, 46)),
 ('mean_dose', (221, 47, 1)),
 ('mean_dose_missing', (221, 47, 1)),
 ('centroids', (221, 47, 3)),
 ('centroids_missing', (221, 47, 3)),
 ('volume', (221, 47, 1)),
 ('volume_missing', (221, 47, 1))]

In [69]:
#check the error on the neural net for inputing/denoising
#only considers non-missing values
#value are before performing clipping on the inputed values
di.error_report()

,_mean_reconstruction_error (%),mean_denoised,mean_original,num_nan_denoised,num_nan_original,num_negative_denoised,num_negative_original,shape_denoised,shape_original,std_denoised,std_original
key,,,,,,,,,,,
distances,7.903354,52.77325,52.61590,0,5732,17074,18250,"(221, 47, 46)","(221, 47, 46)",41.22015,41.38671
mean_dose,66.938509,40.00218,39.29906,0,117,4,0,"(221, 47, 1)","(221, 47, 1)",20.44255,21.81310
centroids,6.605791,171.54075,171.91481,0,204,0,0,"(221, 47, 3)","(221, 47, 3)",82.48657,82.80502
volume,21.497179,16.30311,16.57408,0,164,0,0,"(221, 47, 1)","(221, 47, 1)",25.35475,26.16836


In [70]:
Utils.np_dict_to_json(ddict,Const.denoised_organ_array_dict,False)

True

In [71]:
def extract_pdict_gtvs(p):
    gtvlist = []
    for pid,pentry in p['patients'].items():
        plist = []
        for o,oentry in pentry.items():
            if 'GTV' in o:
                plist.append((o,oentry))
        gtvlist.append((pid, plist))
    return gtvlist

#turn it into another dictionary again
def inputed_data_dict(pdata, insert_gtvs = True):
    pids = Formatting.get_sorted_pids(pdata)
    organs = pdata['organs'] + ['gtv_composite']
    main_keys = [k for k in ddict.keys() if '_missing' not in k]
    mask_keys =[k for k in ddict.keys() if k not in main_keys]
    make_dict = lambda keys: {int(p): {o: {k: np.nan for k in keys} for o in organs} for p in pids}
    inputed_data = make_dict(main_keys)
    data_mask = make_dict(mask_keys)
    for k,varray in ddict.items():
        for pid, v_row in zip(pids,varray):
            for organ, o_col in zip(organs, v_row):
                val = o_col
                if len(val) <= 1:
                    val = val[0]
                if k in mask_keys:
                    data_mask[int(pid)][organ][k.replace('_missing','')] = val
                else:
                    inputed_data[int(pid)][organ][k] = val
    return_dict = {'organs': organs, 'patient_ids': pids, 
            'patients': inputed_data, 'mask': data_mask}
    if insert_gtvs:
        gtvset = extract_pdict_gtvs(pdata)
        for pid,gtvs in gtvset:
            for k,v in gtvs:
                return_dict['patients'][int(pid)][k] = v
    return return_dict

inputed_ddict = inputed_data_dict(pdata)
[(k,v) for k,v in inputed_ddict['patients'][185].items() if 'GTV' in k]

[('GTVp',
  {'volume': 10.8,
   'mean_dose': 71.75,
   'centroids': [279.6041, 198.22, 54.5221],
   'distances': [83.0,
    27.96875,
    41.625,
    82.5,
    59.40625,
    73.375,
    72.5,
    53.40625,
    35.59375,
    20.46875,
    29.28125,
    26.65625,
    27.71875,
    55.0625,
    72.1875,
    53.78125,
    34.5625,
    44.59375,
    57.3125,
    18.390625,
    36.75,
    12.84375,
    -3.97265625,
    -1.953125,
    12.15625,
    19.34375,
    11.59375,
    -4.98046875,
    35.9375,
    10.046875,
    -4.02734375,
    1.953125,
    -3.3203125,
    28.078125,
    2.5,
    6.1171875,
    -3.90625,
    6.90625,
    23.9375,
    56.6875,
    70.9375,
    73.0,
    83.0,
    58.09375,
    58.625,
    49.875]}),
 ('GTVn',
  {'volume': 4.6,
   'mean_dose': 72.28,
   'centroids': [293.0174, 212.0342, 61.6571],
   'distances': [52.1875,
    29.421875,
    6.734375,
    63.90625,
    30.453125,
    39.90625,
    54.6875,
    29.796875,
    14.4921875,
    14.9453125,
    43.40625,
  

In [72]:
#hopefully there will be no nulls, but idk
Utils.np_dict_to_json(inputed_ddict, Const.denoised_organ_json,True)

True

In [ ]:
#Below is code when I was trying to create pointclouds that 
#preserved the spatial features 
#(Does not work)

In [ ]:
from shapely.geometry import Polygon,MultiPolygon

def slice_ddict(pdata, ddict, organs = None):
    if organs is None:
        return {k:v for k,v in ddict.items()}
    olist = pdata['organs']
    indx = [olist.index(o) for o in organs]
    new_dict = {}
    for k,v in ddict.items():
        new_val = v[:,indx + [-1],:]
        if 'distance' in k:
            new_val = new_val[:,:,indx]
        new_dict[k] = new_val
    return new_dict

def radius_from_volume(v, eps = .01):
    if v < eps:
        v = eps
    return (v*3/(4))**(1/3)

def getSphereSize(npoints):
    slices = int(np.sqrt(npoints))
    stops = int(np.ceil(npoints/slices))
    return slices, stops

def sphere3d(center, radius, npoints):
    slices, stops = getSphereSize(npoints)
    #will not necessarily b npoints because its weird
    pcloud = np.empty((slices*stops, 3))
    pcloud[:] = center
    theta_rotation = 2*np.pi/stops
    phi_rotation = np.pi/slices
    phi = 0
    count = 0
    for i in range(slices):
        theta = 0
        phi_vec = np.array([np.sin(phi),np.sin(phi),np.cos(phi)])
        for ii in range(stops):
            theta_vec = np.array([np.cos(theta),np.sin(theta), 1])
            vector = radius * (theta_vec * phi_vec)
            pcloud[count] += vector
            theta += theta_rotation
            count += 1
        phi += phi_rotation
    return pcloud
            

def sphere2d(center, radius, npoints):
    pcloud = np.empty((npoints, 2))
    pcloud[:] = center
    rotation = 2*np.pi/(npoints)
    angle = 0
    for idx in range(npoints):
        vector = [np.cos(angle), np.sin(angle)]
        pcloud[idx] += radius*np.array(vector)
        angle += rotation
    return pcloud

def make_pointcloud(centroid, volume,npoints):
    #shape (ndim,) = (3,) (1,) (norgans,)
    radius = radius_from_volume(volume[0])
    ndims = centroid.shape[0]
    if ndims == 2:
        return sphere2d(centroid,radius,npoints)
    else:
        return sphere3d(centroid,radius,npoints)
    
def make_organset_pointcloud(centroids, volume, distances,npoints):
    #n_organs x n_dim shaped inputs
    n_organs = centroids.shape[0]
    pclouds = []
    for oidx in range(n_organs):
        try:
            organ_pcloud = make_pointcloud(centroids[oidx],
                                           volume[oidx],
                                           npoints)
            pclouds.append(organ_pcloud)
        except Exception as e:
            print('error making single pcloud',e)
            
    pclouds = np.stack(pclouds)
    return pclouds

def polygon_shape_error(pgon,vol,centroid,alpha = .1, beta = 1):
    vol_err = np.abs((vol[0] - pgon.area))/(vol[0])
    center = [pgon.centroid.x,pgon.centroid.y]
    c_err = np.linalg.norm(center - centroid)
    return alpha*vol_err + beta*c_err

def pcloud_loss(pcloud,vols,centroids,dists,vweight=.01,cweight=.5,dweight=1):
    #n_organs + 1 x ndim shaped arrays
    #pcloud (x npoints x ndim), vol (x1), centroids (x ndim) dists(x norgans)
    err = 0
    norgans = dists.shape[1]
    pgons = [Polygon(pc) for pc in pcloud]
    for i,pgon in enumerate(pgons):
        shape_err = polygon_shape_error(pgon,vols[i],centroids[i],vweight,cweight)
        d_err = 0
        for odist, pgon2 in zip(dists[i],pgons):
            d_err += np.abs((pgon.distance(pgon2) - odist)/odist)
        d_err *= dweight/(len(pgons) - 1)
    return d_err + shape_err

def get_starter_pointclouds(pdata, ddict,organs=None,ndim = 2,npoints = None):
    #should in-theory generate an array of polygons for each organ
    #based around centroid
    d = slice_ddict(pdata,ddict, organs)
    pointclouds = []
    n_patients = ddict['centroids'].shape[0]
    vols = d['volume']
    centroids = d['centroids']
    dists = d['distances']
    #do pca to reduce from 3 to 2 dimensions
    assert(ndim in [2,3])
    
    #double check I did this right later
    centroids = centroids - centroids.mean(axis=1).mean(axis=0)
    #so I should probabl do some unit analysis here
    vols = vols.clip(min=.1)#/vols.max(axis=0)
    dists = dists.clip(min=.1)
    if ndim == 2:
        #so this should maybe be somehting idk
        centroids = centroids[:,:,0:2]
#         cshape = centroids.shape
#         c = centroids.reshape((-1,3))
#         print('staring mds')
#         c = manifold.Isomap().fit_transform(c)
#         print('finished')
#         centroids = c.reshape((cshape[0],cshape[1],ndim))
    if npoints is None:
        npoints = 10**(ndim-1)
    for idx in range(n_patients):
        try:
            pcloud = make_organset_pointcloud(centroids[idx],vols[idx],dists[idx],npoints)
            pointclouds.append(pcloud)
        except Exception as e:
            print('error making pointcloud')
            print(e)
    #will be n_patients, n_organs, n_points, n_dimensions
    #so output[x] will be a multipolygon for a patient 
    #and output[x,y] will be a pointcloud for an organ
    return np.stack(pointclouds)
        
#for some reason the masseter is alway way out there
test_organs = [o for o in pdata['organs']]# if "Masset" not in o]
pcs = get_starter_pointclouds(pdata,ddict, ndim = 2,organs=test_organs)

In [ ]:
MultiPolygon([Polygon(p) for pc in pcs[0:1] for p in pc])

In [ ]:
pcs[0,0].var(axis=0)/ddict['volume'][0,0]

In [ ]:
import torch.nn as nn
import torch
from Pytorchtools import EarlyStopping
from SpatialLosses import *
class PointCloudOptimizer(nn.Module):
    
    def __init__(self,
                 norgans,
                 ndim = 2,
                 npoints = 10,
                 hidden_dims = [1000]
                ):
        super(PointCloudOptimizer,self).__init__()
        self.ndim = ndim
        #I have to change the point size to work with spheres evenly
        self.npoints = npoints
        if ndim == 3: 
            i,ii = getSphereSize(npoints)
            self.true_npoints = ii*i
        else:
            self.true_npoints = npoints
        self.norgans = norgans
        self.pcloud_size = int(ndim*npoints)
        input_organ_size = self.pcloud_size + norgans + ndim + 1
        layers = [ nn.Linear(norgans*input_organ_size,hidden_dims[0]) ]
        for i in range(0,len(hidden_dims)-1):
            layers.append(nn.Linear(hidden_dims[i],hidden_dims[i+1]))
            layers.append(nn.BatchNorm1d(hidden_dims[i+1]))
            
        self.output_shape = (norgans,npoints,ndim)
        layers.append(nn.Linear(hidden_dims[-1], norgans*npoints*ndim))
        self.layers = nn.Sequential( *layers )
        self.layers.apply(self.init_weights)
        
    def init_weights(self,layer):
        #make the transform weights zero to start
        #so the output is always a valid polygon when it starts
        if isinstance(layer, nn.Linear):
            nn.init.normal_(layer.weight,mean=0.0,std=0.0)
        
    def make_pclouds(self,x):
        #takes an entry from the centroid, volumes and distances of a patient
        #converts from numpy to torch also
        #c (centroids): norgans x ndim, 
        #v (volume): norgans x 1, 
        #d (distances) norgans x norgans
        [cs,vs,ds] = [i.cpu().detach().numpy() for i in x]
        pclouds = [make_organset_pointcloud(c,v,d,self.npoints) for c,v,d in zip(cs,vs,ds)]
        return torch.tensor(pclouds).float()
        
    def forward(self,x):
        # x is a list [c,v,d] of np arrays
        #c (centroids): len x norgans x ndim, 
        #v (volume): len x norgans x 1, 
        #d (distances) len x norgans x norgans
        
        #makes torch friendly
        #shape len x norgans x true_npoints x ndim
        pointclouds = self.make_pclouds(x)
        pc_shape = pointclouds.shape
        #reshape so it can concatenate with other inputs
        pointclouds = pointclouds.reshape((pc_shape[0],pc_shape[1],-1))
        #now is len x norgans x norgans + ndim + (npoints*ndim)
        out = torch.flatten(torch.cat([pointclouds] + x,dim=2),start_dim=1)
        out = self.layers(out).reshape(pc_shape)
        pointclouds = pointclouds.reshape(pc_shape)
        return out + pointclouds, pointclouds     

def format_pcloud_inputs(ddict):
    oidx = [i for i,o in enumerate(pdata['organs']) if "Masset" not in o]
    v = ddict['volume'][:pcount,oidx].clip(min=.1)
    d = ddict['distances'][:pcount,oidx]
    d = d[:,:,oidx].clip(min=.01)
    #I think volume is in cm3 and distance is mm ?
    d = 10*d
    c = ddict['centroids'][:pcount,oidx,0:ndim]
    c = c - c.mean(axis=0)
    [c, v, d] = [torch.tensor(i).float() for i in [c,v,d]]
    return c, v, d

def train_pcloud_optimizer(ddict, 
                        ndim = 2,
                        npoints = 10,
                        model_path = Const.pytorch_model_dir + 'pcloud.pt',
                        epochs = 1000,
                        lr = .001,
                        centroid_weight = .5,
                        shape_weight = .2,
                        distance_weight = 1
                        ):
    pcount = 20
    c,v,d = format_pcloud_inputs(ddict)

    popt = PointCloudOptimizer(c.shape[1],ndim,npoints = npoints)
    early_stopping = EarlyStopping(patience = 10, path = model_path)
    optimizer = torch.optim.SGD(popt.parameters(),lr =lr)
    loss_hist = []
#     criterion = PointCloudLoss(1,.5)
    for epoch in range(epochs):
        optimizer.zero_grad()
        ypred, ydefault = popt([c,v,d])
        loss = pcloud_loss(ypred,ydefault,
                           c,v,d,
                           shape_weight,centroid_weight,distance_weight)
        loss.backward()
        loss_hist.append(loss.item())
        print('epoch',epoch,loss.item())
        optimizer.step()
        torch.cuda.empty_cache()
        early_stopping(loss.item(),popt)
        if early_stopping.early_stop or torch.isnan(loss):
            print('early stop on epoch',epoch)
            break
    popt.load_state_dict(torch.load(model_path))
    plt.plot(loss_hist)
    return popt.eval(), loss_hist

popt, lh = train_pcloud_optimizer(ddict)

In [ ]:
from shapely.geometry import MultiPoint
pcount = 10
ndim = 2
c,v,d = format_pcloud_inputs(ddict)
ypred,ydefault = popt([c,v,d])
clouds = ypred.cpu().detach().numpy()
MultiPolygon([Polygon(p) for pc in clouds[0:1] for p in pc])

In [ ]:
#the part below is generic notebook analysis

In [ ]:
#checking how many organs are missing in the old dataset
sf = load_spatial_files()
organ_counts = {}
total_patients = 0
for pid, files in sf.items():
    dosefile = files['doses']
    distancefile = files['distances']
    def read_replace(file):
        df = pd.read_csv(file)
        df.replace(to_replace = r'_*GTV.*N', value = '_GTVn', regex = True, inplace = True)
        return df
    dosedf = read_replace(dosefile)
    distdf = read_replace(distancefile)
    dose_rois = set(dosedf['ROI'].dropna())
    dist_rois = set(distdf['Reference ROI'].dropna()).union(set(distdf['Target ROI'].dropna()))
    rois = dose_rois.intersection(dist_rois)
    has_gtv = False
    for r in rois:
        if 'GTV' in r:
            has_gtv = True
    if not has_gtv:
        continue
    else:
        total_patients += 1
    for roi in rois:
        if 'GTV' in roi:
            continue
        currcount = organ_counts.get(roi, 0)
        organ_counts[roi] = currcount + 1
def tempsort(val):
    key = val[0]
    if key in Const.camprt_organs:
        idx = Const.camprt_organs.index(key)
    else:
        idx = 0
    return (.5*val[1]/total_patients) + (2*idx/len(Const.camprt_organs))
values = sorted([(k,v) for k,v in organ_counts.items()], key = tempsort)

In [ ]:
axis = [i for i,v in enumerate(values)]
widths = [x[1] for x in values]
labels = [x[0] for x in values]
colors = []
for label, val in values:
    if label in Const.camprt_organs:
        if val >= total_patients:
            c ='blue'
        else:
            c = 'cyan'
    else:
        c = 'red'
    colors.append(c)
plt.rcParams['figure.figsize'] = [10,30]
fix, ax = plt.subplots()
ax.barh(axis,widths,
         tick_label=labels,
        color = colors,
        log = False)
ax.plot([total_patients,total_patients],[0,len(axis)],"k--")

In [ ]:
np.linalg.norm([1,1,1])